We choose pyumpf and check the documentation: https://pymupdf.readthedocs.io/en/latest/pymupdf4llm/

In [ ]:
%pip install pymupdf4llm

We then try to convert a pdf document to markdown

In [3]:

import pymupdf4llm
md_text = pymupdf4llm.to_markdown("Bundesverfassung der Schweizerischen Eidgenossenschaft.pdf")


In [4]:
md_text

'**Bundesverfassung**\n**der Schweizerischen Eidgenossenschaft**\n\nvom 18. April 1999 (Stand am 3. März 2024)\n\n**Präambel**\n\nIm Namen Gottes des Allmächtigen!\n\n*Das Schweizervolk und die Kantone,*\n\nin der Verantwortung gegenüber der Schöpfung,\n\n\n**101**\n\n\nim Bestreben, den Bund zu erneuern, um Freiheit und Demokratie, Unabhängigkeit\nund Frieden in Solidarität und Offenheit gegenüber der Welt zu stärken,\n\nim Willen, in gegenseitiger Rücksichtnahme und Achtung ihre Vielfalt in der Einheit\nzu leben,\n\nim Bewusstsein der gemeinsamen Errungenschaften und der Verantwortung gegenüber den künftigen Generationen,\n\ngewiss, dass frei nur ist, wer seine Freiheit gebraucht, und dass die Stärke des Volkes\nsich misst am Wohl der Schwachen,\n\n*geben sich folgende Verfassung* [1] *:*\n\n**1. Titel: Allgemeine Bestimmungen**\n\n**Art. 1** Schweizerische Eidgenossenschaft\n\nDas Schweizervolk und die Kantone Zürich, Bern, Luzern, Uri, Schwyz, Obwalden\nund Nidwalden, Glarus, Zug, 

In [6]:
import pathlib
pathlib.Path("Bundesverfassung_der_Schweizerischen_Eidgenossenschaft.md").write_bytes(md_text.encode())

244548

It works so we do the rest of the documents as well

In [8]:
Arbeitsgesetz=pymupdf4llm.to_markdown("Arbeitsgesetz.pdf")
pathlib.Path("Arbeitsgesetz.md").write_bytes(Arbeitsgesetz.encode())

69679

In [11]:
from langchain_community.document_loaders import PyMuPDFLoader
loader = PyMuPDFLoader("OR.pdf")
OR = loader.load()
OR

[Document(metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2023-06-07T11:11:19+02:00', 'source': 'OR.pdf', 'file_path': 'OR.pdf', 'total_pages': 552, 'format': 'PDF 1.7', 'title': '220', 'author': 'Schweizerische Bundeskanzlei', 'subject': 'Bundesgesetz betreffend die Ergänzung  des Schweizerischen Zivilgesetzbuches  (Fünfter Teil: Obligationenrecht)', 'keywords': 'Rechtstext Formatvorlage DfV', 'moddate': '2023-06-07T11:11:19+02:00', 'trapped': '', 'modDate': "D:20230607111119+02'00'", 'creationDate': "D:20230607111119+02'00'", 'page': 0}, page_content='1 / 552 \nBundesgesetz \nbetreffend die Ergänzung  \ndes Schweizerischen Zivilgesetzbuches  \n(Fünfter Teil: Obligationenrecht) \n \nvom 30. März 1911 (Stand am 9. Februar 2023) \n \nDie Bundesversammlung der Schweizerischen Eidgenossenschaft, \nnach Einsicht in die Botschaften des Bundesrates vom 3. März 1905 und  \n1. Juni 19091, \nbeschliesst: \nErste Abteilung: Allgemeine Bestimmunge

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Split into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(OR)

# Inspect one chunk to verify
print(chunks[0].page_content)


1 / 552 
Bundesgesetz 
betreffend die Ergänzung  
des Schweizerischen Zivilgesetzbuches  
(Fünfter Teil: Obligationenrecht) 
 
vom 30. März 1911 (Stand am 9. Februar 2023) 
 
Die Bundesversammlung der Schweizerischen Eidgenossenschaft, 
nach Einsicht in die Botschaften des Bundesrates vom 3. März 1905 und  
1. Juni 19091, 
beschliesst: 
Erste Abteilung: Allgemeine Bestimmungen 
Erster Titel: Die Entstehung der Obligationen 
Erster Abschnitt: Die Entstehung durch Vertrag 
Art. 1


In [14]:
%pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Feresh\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [16]:
%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Feresh\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [19]:
from langchain.embeddings import HuggingFaceEmbeddings

# Create embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Embed a sample chunk
vector_example = embeddings.embed_query(chunks[0].page_content)
print(vector_example[:5])  # Check first few embedding dimensions


[-0.04195893555879593, 0.07454517483711243, 0.04941004142165184, -0.059933602809906006, -0.02894851751625538]


In [22]:
%pip install --upgrade pip

  Using cached pip-25.1.1-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 21.2.3
    Uninstalling pip-21.2.3:
      Successfully uninstalled pip-21.2.3
Note: you may need to restart the kernel to use updated packages.


In [24]:
%pip install faiss-cpu


   ---------------------------------------- 0.0/15.0 MB ? eta -:--:--
    --------------------------------------- 0.3/15.0 MB ? eta -:--:--
   -- ------------------------------------- 1.0/15.0 MB 3.9 MB/s eta 0:00:04
   --- ------------------------------------ 1.3/15.0 MB 2.3 MB/s eta 0:00:06
   ---- ----------------------------------- 1.8/15.0 MB 2.7 MB/s eta 0:00:05
   ------ --------------------------------- 2.6/15.0 MB 2.9 MB/s eta 0:00:05
   -------- ------------------------------- 3.1/15.0 MB 2.6 MB/s eta 0:00:05
   --------- ------------------------------ 3.7/15.0 MB 2.6 MB/s eta 0:00:05
   ------------ --------------------------- 4.7/15.0 MB 3.0 MB/s eta 0:00:04
   -------------- ------------------------- 5.5/15.0 MB 3.0 MB/s eta 0:00:04
   ------------------ --------------------- 6.8/15.0 MB 3.4 MB/s eta 0:00:03
   -------------------- ------------------- 7.9/15.0 MB 3.6 MB/s eta 0:00:02
   ----------------------- ---------------- 8.9/15.0 MB 3.8 MB/s eta 0:00:02
   ----------

In [25]:
from langchain.vectorstores import FAISS

# Create and save the vectorstore locally
vectorstore = FAISS.from_documents(chunks, embeddings)
vectorstore.save_local("OR_vectorstore")


In [35]:
%pip install transformers accelerate


Note: you may need to restart the kernel to use updated packages.


In [27]:
%pip install optimum


Note: you may need to restart the kernel to use updated packages.


In [28]:
%pip install transformers accelerate bitsandbytes



   ---------------------------------------- 0.0/75.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/75.4 MB ? eta -:--:--
   ---------------------------------------- 0.3/75.4 MB ? eta -:--:--
   ---------------------------------------- 0.3/75.4 MB ? eta -:--:--
   ---------------------------------------- 0.8/75.4 MB 1.2 MB/s eta 0:01:05
    --------------------------------------- 1.6/75.4 MB 2.0 MB/s eta 0:00:37
   - -------------------------------------- 2.1/75.4 MB 2.3 MB/s eta 0:00:32
   - -------------------------------------- 3.1/75.4 MB 2.8 MB/s eta 0:00:27
   - -------------------------------------- 3.7/75.4 MB 2.8 MB/s eta 0:00:26
   -- ------------------------------------- 4.7/75.4 MB 3.0 MB/s eta 0:00:24
   -- ------------------------------------- 5.5/75.4 MB 3.1 MB/s eta 0:00:23
   --- ------------------------------------ 6.6/75.4 MB 3.3 MB/s eta 0:00:22
   --- ------------------------------------ 7.1/75.4 MB 3.3 MB/s eta 0:00:21
   ---- ------------------

In [29]:
%pip install auto-gptq optimum


  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached pandas-2.2.3-cp310-cp310-win_amd64.whl.metadata (19 kB)
  Using cached xxhash-3.5.0-cp310-cp310-win_amd64.whl.metadata (13 kB)
  Using cached multiprocess-0.70.16-py310-none-any.whl.metadata (7.2 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/4.6 MB ? eta -:--:--
   ---- ----------------------------------- 0.5/4.6 MB 5.6 MB/s eta 0:00:01
   ------------- -------------------------- 1.6/4.6 MB 4.4 MB/s eta 0:00:01
   ------------------ --------------------- 2.1/4.6 MB 4.3 MB/s eta 0:00:01
   -------------------- ------------------- 2.4/4.6 MB 3.4 MB/s eta 0:00:01
   --------------------------- ------------ 3.1/4.6 MB 3.5 MB/s eta 0:00:01
   ------------------------------------ --- 4.2/4.6 MB 3.7 MB/s eta 0:00:01
   ---------------------------------------- 4.6/4.6 MB 3.8 MB/s eta

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
from huggingface_hub import login

token = os.getenv("HUGGINGFACE_TOKEN")
login(token)




In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_id = "mistralai/Mistral-7B-Instruct-v0.2"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",           # use GPU if available, CPU otherwise
    torch_dtype="auto"           # supports bf16/float16 if available
)

# Create pipeline
mistral_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)


c:\Users\Feresh\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\models\auto\tokenization_auto.py:898: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

c:\Users\Feresh\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Feresh\.cache\huggingface\hub\models--mistralai--Mistral-7B-Instruct-v0.2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Device set to use cpu
